In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

!pip install -q tensorflow==2.0.0-beta1
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Weather.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [3]:
df.fillna(method='ffill', inplace=True)

In [5]:
df.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RISK_MM          float64
RainTomorrow      object
dtype: object

In [6]:
df['Date'] = pd.Categorical(df['Date'])
df['Date'] = df.Date.cat.codes

df['Location'] = pd.Categorical(df['Location'])
df['Location'] = df.Location.cat.codes

df['WindGustDir'] = pd.Categorical(df['WindGustDir'])
df['WindGustDir'] = df.WindGustDir.cat.codes

df['WindDir9am'] = pd.Categorical(df['WindDir9am'])
df['WindDir9am'] = df.WindDir9am.cat.codes

df['WindDir3pm'] = pd.Categorical(df['WindDir3pm'])
df['WindDir3pm'] = df.WindDir3pm.cat.codes

df['RainToday'] = pd.Categorical(df['RainToday'])
df['RainToday'] = df.RainToday.cat.codes

df['RainTomorrow'] = pd.Categorical(df['RainTomorrow'])
df['RainTomorrow'] = df.RainTomorrow.cat.codes

In [7]:
df.fillna(df.mean(), inplace=True)
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,396,2,13.4,22.9,0.6,6.329457,7.056243,13,44.0,13,...,22.0,1007.7,1007.1,8.0,4.62044,16.9,21.8,0,0.0,0
1,397,2,7.4,25.1,0.0,6.329457,7.056243,14,44.0,6,...,25.0,1010.6,1007.8,8.0,4.62044,17.2,24.3,0,0.0,0
2,398,2,12.9,25.7,0.0,6.329457,7.056243,15,46.0,13,...,30.0,1007.6,1008.7,8.0,2.00000,21.0,23.2,0,0.0,0
3,399,2,9.2,28.0,0.0,6.329457,7.056243,4,24.0,9,...,16.0,1017.6,1012.8,8.0,2.00000,18.1,26.5,0,1.0,0
4,400,2,17.5,32.3,1.0,6.329457,7.056243,13,41.0,1,...,33.0,1010.8,1006.0,7.0,8.00000,17.8,29.7,0,0.2,0


In [8]:
train, test = train_test_split(df, test_size=0.2)

print( 'train: ',len(train))
print('test: ',len(test))

traint= train.pop('RainTomorrow')
testt= test.pop('RainTomorrow')

train:  113754
test:  28439


In [9]:
dataset = tf.data.Dataset.from_tensor_slices((train.values, traint.values))
train_dataset = dataset.shuffle(len(train)).batch(1)

In [10]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [11]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Epoch 1/15


W0724 23:07:21.616348 20452 deprecation.py:323] From C:\Users\Ramesh\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


113754/113754 [==============================] - 162s 1ms/step - loss: 3.4478 - accuracy: 0.7747
Epoch 2/15
113754/113754 [==============================] - 176s 2ms/step - loss: 3.4429 - accuracy: 0.7771
Epoch 3/15
113754/113754 [==============================] - 189s 2ms/step - loss: 3.4429 - accuracy: 0.7771
Epoch 4/15
113754/113754 [==============================] - 185s 2ms/step - loss: 3.4429 - accuracy: 0.7771
Epoch 5/15
113754/113754 [==============================] - 181s 2ms/step - loss: 3.4429 - accuracy: 0.7771
Epoch 6/15
113754/113754 [==============================] - 178s 2ms/step - loss: 3.4429 - accuracy: 0.7771
Epoch 7/15
113754/113754 [==============================] - 187s 2ms/step - loss: 3.4429 - accuracy: 0.7771
Epoch 8/15
113754/113754 [==============================] - 179s 2ms/step - loss: 3.4429 - accuracy: 0.7771
Epoch 9/15
113754/113754 [==============================] - 176s 2ms/step - loss: 3.4429 - accuracy: 0.7771
Epoch 10/15
113754/113754 [============

In [12]:
testdta = tf.data.Dataset.from_tensor_slices((test.values, testt.values))
testdta = testdta.shuffle(len(test)).batch(1)

In [13]:
loss, accuracy = model.evaluate(testdta)
print("Accuracy: ", accuracy)
print("Loss: ",loss)

28439/28439 [==============================] - 21s 746us/step - loss: 3.5185 - accuracy: 0.7719
Accuracy:  0.77189773
Loss:  3.5184655637361146
